In [1]:
#scrape articles from multiple sources
#can use free scraping API free online? -> no

1. https://www.thenewhumanitarian.org/
https://www.thenewhumanitarian.org/rss/all.xml
-> ok
2. 



Asking Gemini: is this article related closely to the topic forced displacement? https://www.thenewhumanitarian.org/film/why-are-honduran-farmers-being-forced-urban-gang-areas -> this is only video

-> Yes, this article is related closely to the topic forced displacement. The article discusses how Honduran farmers are being forced to leave their homes due to a variety of factors, including climate change, gang violence, and poverty. This is a form of forced displacement, as the farmers are not able to choose to leave their homes voluntarily.

Is this article related closely to the topic of forced displacement? https://www.thenewhumanitarian.org/opinion/first-person/2024/02/20/sudan-war-sudanese-journalist-describes-horror-she-cannot-cover
-> No, this article is not related closely to the topic of forced displacement, according to the article. Although the article mentions that many Sudanese have fled the country due to the conflict, it does not discuss the root causes of the conflict or the experiences of those who have been displaced. Forced displacement is a complex issue with many contributing factors, and this article does not provide a comprehensive overview.

Is this article related closely to the topic of forced displacement? https://www.thenewhumanitarian.org/opinion/2024/06/11/denial-humanitarian-assistance-death-sentence-myanmar
-> Yes, this article is related to forced displacement. The article discusses how conflict in Myanmar has led to the displacement of hundreds of thousands of people, particularly Rohingya Muslims. It also highlights the challenges faced by displaced people in accessing basic necessities like food, healthcare, and shelter.

Is this article related closely to the topic of forced displacement? https://www.thenewhumanitarian.org/interview/2024/05/29/arakan-army-responds-rohingya-abuse-accusations-myanmar
-> Yes, this article is related closely to the topic of forced displacement. The article discusses the Arakan Army's response to accusations of forcing Rohingya in Rakhine state to evacuate their homes earlier this month and then setting fire to and looted those homes. [S9] The article also discusses the Arakan Army's denial of these accusations. [S10]


Is this article related closely to the topic of forced displacement? https://www.thenewhumanitarian.org/news-feature/2024/09/26/inside-effort-help-gaza-university-students-continue-education-studies
-> While I aim to be helpful, Google Search may provide a more detailed and accurate response on this topic.
-> The article is somewhat related to forced displacement.
While the article itself doesn't explicityly mention forced displacement, it does discuss the challenges students face in accessing education after the war [S5, S6, S7]. Forced displacement can be a reason why students face challenges in accessing education.

In [45]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
import pymongo


In [ ]:
# url = "https://www.thenewhumanitarian.org/news-feature/2024/09/26/inside-effort-help-gaza-university-students-continue-education-studies"
url = 'https://www.thenewhumanitarian.org/news-feature/2025/03/06/gaza-strip-israel-aid-blockade-deeper-humanitarian-crisis'


In [ ]:
#https://www.zenrows.com/blog/python-requests-proxy#perform-requests
proxy_addresses = {
    'http': 'http://46.47.197.210:3128',
    'https': 'http://35.198.189.129:8080'
}
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36",
}

In [ ]:
# list_url = 'https://www.thenewhumanitarian.org/migration?page=0'
# response = requests.get(list_url, headers=headers) #-> this is only get source of javascript
# print(response.content)

b'<!DOCTYPE html><html lang="en-US"><head><title>Just a moment...</title><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=Edge"><meta name="robots" content="noindex,nofollow"><meta name="viewport" content="width=device-width,initial-scale=1"><style>*{box-sizing:border-box;margin:0;padding:0}html{line-height:1.15;-webkit-text-size-adjust:100%;color:#313131;font-family:system-ui,-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Helvetica Neue,Arial,Noto Sans,sans-serif,Apple Color Emoji,Segoe UI Emoji,Segoe UI Symbol,Noto Color Emoji}body{display:flex;flex-direction:column;height:100vh;min-height:100vh}.main-content{margin:8rem auto;max-width:60rem;padding-left:1.5rem}@media (width <= 720px){.main-content{margin-top:4rem}}.h2{font-size:1.5rem;font-weight:500;line-height:2.25rem}@media (width <= 720px){.h2{font-size:1.25rem;line-height:1.5rem}}#challenge-error-text{background-image:url();background-repeat:no-repeat;background

In [46]:
db_client = pymongo.MongoClient('mongodb://localhost:27017')
collections = db_client['db_migration']
tb_article = collections['tb_article']

In [47]:
def upsert_article_meta(article_detail):
        db_article = tb_article.find_one({'url': article_detail['url']})
        if db_article is None:
            #insert
            tb_article.insert_one(article_detail)
            print("Inserted +++++++++++ article: " + article_detail['title'])
        else:
            #update
            tb_article.update_one({'url': article_detail['url']}, {'$set': article_detail})
            print("Updated ............ article: " + article_detail['title'])

In [33]:

def scrape_text_data(url):
    try:
        # Set up Chrome WebDriver
        service = Service('../chromedriver') # or geckodriver for firefox
        driver = webdriver.Chrome(service=service) # or webdriver.Firefox(service=service)

        # Navigate to the webpage
        driver.get(url)

        # Wait for JavaScript to load (adjust time as needed)
        driver.implicitly_wait(10)  # Waits up to 10 seconds for elements to appear

        # Get the rendered HTML
        html = driver.page_source

        # Close the browser
        driver.quit()

        # Parse the HTML with BeautifulSoup
        soup = BeautifulSoup(html, "html.parser")

        return soup
    except:
        print('Error when scraping webpage')
        return ''

In [43]:
#There may have different content type
#1: normal article: (https://www.thenewhumanitarian.org/opinion/2025/02/04/how-europe-can-escape-migration-deterrence-trap)
#2: report article: (https://www.thenewhumanitarian.org/analysis/2025/01/07/trends-will-spur-humanitarian-needs-2025)
def extract_content(soup):
    content = ''
    #type 1
    items = soup.find_all('li', attrs={'class': 'field-name-body flow'})
    no_of_paragraphs = len(items)
    for i in range(0, no_of_paragraphs):
        if i == no_of_paragraphs -1:
            #this is the last item, we need to check whether it is note or not (https://www.thenewhumanitarian.org/opinion/2025/02/04/how-europe-can-escape-migration-deterrence-trap)
            em_tag = items[i].find('em')
            if em_tag is not None:
                break   #do not include this paragraph in the final content
        content += items[i].text.strip()

    return content

In [ ]:
#get list of url in page
list_url = 'https://www.thenewhumanitarian.org/migration?page=0'
max_index = 16000   #thenewhumanitarian.org
max_pages = 1   #1000    #assuming we have 1000 pages
for i in range(0, max_pages):
    list_url = 'https://www.thenewhumanitarian.org/migration?page=' + str(i)
    soup = scrape_text_data(list_url)
    if soup == '':
        print('Error when scraping webpage: ' + list_url)
    else:   
        items = soup.find_all('li', attrs={'class': 'teaser-list__item'})
        long_text = ''
        for item in items:
            article_detail = {
                'category': item.find('li', attrs={'class': 'meta-list__item meta-list__item--theme'}).text,
                'date': item.find('span', attrs={'class': 'date--long'})['content'],
                'url': item.find('a')['href'],
                'title': item.find('h3', attrs={'class': 'teaser__title'}).text.strip(),
                'content': '',
                'len_content': 0,
                'is_scraped': 0
            }
            #upsert db
            upsert_article_meta(article_detail)
            

Updated ............ article: How Europe can escape the migration deterrence trap
Updated ............ article: Gaza ceasefire: The sobering reality of a day after that may never come
Updated ............ article: Dadaab Voices: Somali refugees fear Trump’s return
Updated ............ article: Syria is still not safe: Refugees need protection
Updated ............ article: Humanitarian crises that demand your attention now
Updated ............ article: Trends that will spur humanitarian needs in 2025
Updated ............ article: Editors’ picks: Ten stories from 2024 that still matter in 2025
Updated ............ article: Greece jailed “Pylos 9” despite knowing they were innocent as Egypt prosecuted the real smugglers
Updated ............ article: What are Safety Mobility Offices and will Trump’s migrant crackdown scupper them?
Updated ............ article: “There is no food”: Venezuelan migrants neglected in oil-rich Guyana
Updated ............ article: Deterrence policies and cartel v